In [9]:
!pip install ipdb
!pip install transformers

!pip install optuna
!pip install pyspellchecker

     |████████████████████████████████| 1.9MB 293kB/s eta 0:00:01


In [2]:
# !git clone https://github.com/plasticityai/magnitude.git
# %cd magnitude/
# !python setup.py install

# Package loading 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymagnitude import *

from sklearn.metrics import accuracy_score, precision_score
from sklearn.metrics import log_loss

import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from keras.preprocessing.text import Tokenizer

[nltk_data] Downloading package punkt to /home/aims/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/aims/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/aims/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/aims/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
Using TensorFlow backend.


# Data Exploration

In [3]:
train = pd.read_csv("./data/Train.csv")
test = pd.read_csv("./data/Test.csv")

sample = pd.read_csv("./data/SampleSubmission.csv")

In [4]:
train.tail()

,ID,text,label
611,BOHSNXCN,What should I do to stop alcoholism?,Alcohol
612,GVDXRQPY,How to become my oldself again,Suicide
613,IO4JHIQS,How can someone stop it?,Alcohol
614,1DS3P1XO,I feel unworthy,Depression
615,ORF71PVQ,I feel so discouraged with life,Depression


In [7]:
train.text.values[:50]

array(['I feel that it was better I dieAm happy',
       'Why do I get hallucinations?',
       'I am stresseed due to lack of financial support in school',
       'Why is life important?',
       'How could I be helped to go through the depression?',
       'What are the effects of depression to ones health',
       'Why is everything so hard to deal with in this life',
       ' I feel emotionally overwhelmed ',
       'How to manage taking alcohol?',
       'Is heaven open for us who smoke bhang?',
       'How to survive without money?',
       'How do i stop this? What do I do when life becomes unbearable?',
       'I was ever sad,alone and always on drugsNow I know how to cope with negative aspects in my life',
       'I feel empty,absent minded',
       'I experienced depression in my studies',
       'I feel like am alone in this world',
       'I feel low and out of place',
       'What should i do to get out of depression?',
       'How can I stop using alcohol?',
       'I fee

Let check the distribution of the data. 

In [6]:
test.tail()

,ID,text
304,Z9A6ACLK,Yes
305,ZDUOIGKN,My girlfriend dumped me
306,ZHQ60CCH,How can I go back to being my old self?
307,ZVIJMA4O,Is it true bhang is medicinal?
308,ZYIFAY98,how can I overcome the problem?


# Data Preprocessing

In [7]:
", ".join(stopwords.words('english'))

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [8]:
contraction_mapping = {"ain't": "is not", "aren't": "are not",
                       "can't": "cannot", "'cause": "because", "could've": "could have", 
                       "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", 
                       "don't": "do not", "hadn't": "had not", "hasn't": "has not", 
                       "haven't": "have not", "he'd": "he would","he'll": "he will", 
                       "he's": "he is", "how'd": "how did", "how'd'y": "how do you", 
                       "how'll": "how will", "how's": "how is",  "I'd": "I would", 
                       "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have",
                       "I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", 
                       "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", 
                       "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": 
                       "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", 
                       "ma'am": "madam", "mayn't": "may not", "might've": "might have",
                       "mightn't": "might not","mightn't've": "might not have", "must've": "must have", 
                       "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", 
                       "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", 
                       "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", 
                       "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", 
                       "she'll": "she will", "she'll've": "she will have", "she's": "she is", 
                       "should've": "should have", "shouldn't": "should not", 
                       "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
                       "this's": "this is","that'd": "that would", "that'd've": "that would have", 
                       "that's": "that is", "there'd": "there would", "there'd've": "there would have", 
                       "there's": "there is", "here's": "here is","they'd": "they would", 
                       "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", 
                       "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", 
                       "we'd": "we would", "we'd've": "we would have", "we'll": "we will", 
                       "we'll've": "we will have", "we're": "we are", "we've": "we have", 
                       "weren't": "were not", "what'll": "what will", "what'll've": "what will have", 
                       "what're": "what are",  "what's": "what is", "what've": "what have", 
                       "when's": "when is", "when've": "when have", "where'd": "where did", 
                       "where's": "where is", "where've": "where have", "who'll": "who will", 
                       "who'll've": "who will have", "who's": "who is", "who've": "who have", 
                       "why's": "why is", "why've": "why have", "will've": "will have", 
                       "won't": "will not", "won't've": "will not have", "would've": "would have", 
                       "wouldn't": "would not", "wouldn't've": "would not have", 
                       "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have",
                       "y'all're": "you all are","y'all've": "you all have","you'd": "you would", 
                       "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", 
                       "you're": "you are", "you've": "you have" }

In [9]:
import re

def clean_caract(sentence):
    sentence = re.sub(r"[^a-zA-Z0-9?,'’‘´`]+", ' ', sentence).strip()
    sentence = re.sub(r"[,]+", ' , ', sentence).strip()
#     sentence = re.sub(r"[,]+", ' ', sentence).strip()

    sentence = re.sub("([A-Z])", " \\1", sentence).strip() # add space basedon capital later
    sentence = re.sub(r"[?]+", ' ? ', sentence).strip()
#     sentence = re.sub(r"[?]+", ' ', sentence).strip()

    return sentence

In [10]:
train.text[0]

'I feel that it was better I dieAm happy'

In [11]:
clean_caract(train.text[0])

'I feel that it was better  I die Am happy'

In [12]:
train["text_processed"] = train["text"].apply(lambda x: clean_caract(x))
test["text_processed"] = test["text"].apply(lambda x: clean_caract(x))

In [13]:
train['Changed'] = np.where(train['text']==train['text_processed'], 'no', 'yes')
print("{} of {} ({:.4f}%) questions have been changed.".format(len(train[train['Changed']=='yes']), 
                                    len(train), 100*len(train[train['Changed']=='yes'])/len(train)))

test['Changed'] = np.where(test['text']==test['text_processed'], 'no', 'yes')
print("{} of {} ({:.4f}%) questions have been changed.".format(len(test[test['Changed']=='yes']), 
                                    len(test), 100*len(test[test['Changed']=='yes'])/len(test)))

393 of 616 (63.7987%) questions have been changed.
207 of 309 (66.9903%) questions have been changed.


In [14]:
train["text_lower"] = train["text_processed"].str.lower()
test["text_lower"] = test["text_processed"].str.lower()

In [15]:
train['Changed'] = np.where(train['text_processed']==train['text_lower'], 'no', 'yes')
print("{} of {} ({:.4f}%) questions have been changed.".format(len(train[train['Changed']=='yes']), 
                                    len(train), 100*len(train[train['Changed']=='yes'])/len(train)))

test['Changed'] = np.where(test['text_processed']==test['text_lower'], 'no', 'yes')
print("{} of {} ({:.4f}%) questions have been changed.".format(len(test[test['Changed']=='yes']), 
                                    len(test), 100*len(test[test['Changed']=='yes'])/len(test)))

584 of 616 (94.8052%) questions have been changed.
285 of 309 (92.2330%) questions have been changed.


In [16]:
def clean_contractions(text, mapping=contraction_mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text

In [17]:
train["text_contraction"] = train["text_lower"].apply(lambda x: clean_contractions(x))
test["text_contraction"] = test["text_lower"].apply(lambda x: clean_contractions(x))

In [18]:
train['Changed'] = np.where(train['text_contraction']==train['text_lower'], 'no', 'yes')
print("{} of {} ({:.4f}%) questions have been changed.".format(len(train[train['Changed']=='yes']), 
                                    len(train), 100*len(train[train['Changed']=='yes'])/len(train)))

test['Changed'] = np.where(test['text_contraction']==test['text_lower'], 'no', 'yes')
print("{} of {} ({:.4f}%) questions have been changed.".format(len(test[test['Changed']=='yes']), 
                                    len(test), 100*len(test[test['Changed']=='yes'])/len(test)))

15 of 616 (2.4351%) questions have been changed.
6 of 309 (1.9417%) questions have been changed.


In [12]:
from spellchecker import SpellChecker

spell = SpellChecker()
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)
        
# text = 'What are the effects of bhang?'
text = "speling correctin"
correct_spellings(text)
# correct_spellings(clean_contractions(clean_caract(text)))

'spelling correction'

In [20]:
train["text_spellchecker"] = train["text_contraction"].apply(lambda x: correct_spellings(x))
test["text_spellchecker"] = test["text_contraction"].apply(lambda x: correct_spellings(x))

In [21]:
train['Changed'] = np.where(train['text_spellchecker']==train['text_contraction'], 'no', 'yes')
print("{} of {} ({:.4f}%) questions have been changed.".format(len(train[train['Changed']=='yes']), 
                                    len(train), 100*len(train[train['Changed']=='yes'])/len(train)))

test['Changed'] = np.where(test['text_spellchecker']==test['text_contraction'], 'no', 'yes')
print("{} of {} ({:.4f}%) questions have been changed.".format(len(test[test['Changed']=='yes']), 
                                    len(test), 100*len(test[test['Changed']=='yes'])/len(test)))

254 of 616 (41.2338%) questions have been changed.
144 of 309 (46.6019%) questions have been changed.


In [30]:
from collections import Counter
vocab = Counter()
for text in train.text_spellchecker:
    for word in text.split():
        vocab[word] += 1

In [31]:
len(vocab)

802

In [32]:
vocab.most_common(800)

[('i', 439),
 ('?', 209),
 ('to', 186),
 ('how', 169),
 ('feel', 141),
 ('the', 125),
 ('of', 122),
 ('and', 120),
 ('can', 101),
 ('is', 89),
 ('what', 86),
 ('do', 84),
 ('alcohol', 84),
 ('my', 81),
 (',', 76),
 ('am', 64),
 ('stop', 54),
 ('depression', 52),
 ('in', 51),
 ('life', 50),
 ('with', 49),
 ('why', 46),
 ('a', 43),
 ('it', 41),
 ('like', 39),
 ('me', 38),
 ('are', 33),
 ('was', 29),
 ('for', 28),
 ('depressed', 27),
 ('not', 27),
 ('avoid', 26),
 ('get', 25),
 ('taking', 24),
 ('one', 24),
 ('overcome', 24),
 ('low', 23),
 ('feeling', 23),
 ('be', 22),
 ('hang', 22),
 ('sad', 22),
 ('effects', 20),
 ('so', 20),
 ('that', 19),
 ('now', 19),
 ('have', 19),
 ('suicide', 19),
 ('lonely', 19),
 ('from', 18),
 ('better', 17),
 ('should', 17),
 ('when', 16),
 ('out', 16),
 ('myself', 16),
 ('up', 16),
 ('deal', 15),
 ('this', 15),
 ('alone', 15),
 ('does', 15),
 ('weed', 14),
 ('will', 14),
 ('on', 13),
 ('stress', 13),
 ('bad', 13),
 ('causes', 13),
 ('could', 12),
 ('no', 12)

In [33]:
vocab_text = Counter()

for text in test.text_spellchecker:
    for word in text.split():
        if word not in vocab.keys():
            vocab_text[word] += 1

In [34]:
len(vocab_text)

195

In [39]:
sorted(vocab_text.most_common(200), reverse=True)

[('yes', 1),
 ('year', 1),
 ('wrong', 2),
 ('worthy', 1),
 ('worthlessness', 1),
 ('withstand', 1),
 ('wise', 1),
 ('whether', 2),
 ('wellwishers', 1),
 ('wake', 1),
 ('view', 1),
 ('vengeance', 1),
 ('users', 1),
 ('urge', 1),
 ('university', 1),
 ('unit', 1),
 ('unhappy', 2),
 ('unfaithful', 1),
 ('understands', 1),
 ('uncared', 1),
 ('ugly', 1),
 ('trying', 2),
 ('trust', 2),
 ('traumatized', 1),
 ('trauma', 1),
 ('tough', 1),
 ('than', 1),
 ('terribly', 1),
 ('terminate', 1),
 ('temptations', 1),
 ('telling', 1),
 ('tasted', 1),
 ('tackle', 1),
 ('sustain', 1),
 ('successful', 1),
 ('stressing', 1),
 ('strength', 2),
 ('street', 1),
 ('strange', 1),
 ('stagnated', 1),
 ('sort', 1),
 ('something', 1),
 ('somehow', 1),
 ('small', 1),
 ('sleepless', 1),
 ('sleeping', 1),
 ('shit', 1),
 ('session', 1),
 ('seemed', 1),
 ('scared', 1),
 ('saved', 1),
 ('ring', 1),
 ('retake', 1),
 ('resume', 1),
 ('restless', 1),
 ('resentment', 1),
 ('replace', 1),
 ('repeat', 1),
 ('releases', 1),
 ('r

In [22]:
train.head()

,ID,text,label,text_processed,Changed,text_lower,text_contraction,text_spellchecker
0,SUAVK39Z,I feel that it was better I dieAm happy,Depression,I feel that it was better I die Am happy,yes,i feel that it was better i die am happy,i feel that it was better i die am happy,i feel that it was better i die am happy
1,9JDAGUV3,Why do I get hallucinations?,Drugs,Why do I get hallucinations ?,yes,why do i get hallucinations ?,why do i get hallucinations ?,why do i get hallucinations ?
2,419WR1LQ,I am stresseed due to lack of financial suppor...,Depression,I am stresseed due to lack of financial suppor...,yes,i am stresseed due to lack of financial suppor...,i am stresseed due to lack of financial suppor...,i am stressed due to lack of financial support...
3,6UY7DX6Q,Why is life important?,Suicide,Why is life important ?,no,why is life important ?,why is life important ?,why is life important ?
4,FYC0FTFB,How could I be helped to go through the depres...,Depression,How could I be helped to go through the depre...,yes,how could i be helped to go through the depre...,how could i be helped to go through the depre...,how could i be helped to go through the depres...


In [23]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

train["text_stop"] = train["text_spellchecker"].apply(lambda text: remove_stopwords(text))
test["text_stop"] = test["text_spellchecker"].apply(lambda text: remove_stopwords(text))

train.head()

,ID,text,label,text_processed,Changed,text_lower,text_contraction,text_spellchecker,text_stop
0,SUAVK39Z,I feel that it was better I dieAm happy,Depression,I feel that it was better I die Am happy,yes,i feel that it was better i die am happy,i feel that it was better i die am happy,i feel that it was better i die am happy,feel better die happy
1,9JDAGUV3,Why do I get hallucinations?,Drugs,Why do I get hallucinations ?,yes,why do i get hallucinations ?,why do i get hallucinations ?,why do i get hallucinations ?,get hallucinations ?
2,419WR1LQ,I am stresseed due to lack of financial suppor...,Depression,I am stresseed due to lack of financial suppor...,yes,i am stresseed due to lack of financial suppor...,i am stresseed due to lack of financial suppor...,i am stressed due to lack of financial support...,stressed due lack financial support school
3,6UY7DX6Q,Why is life important?,Suicide,Why is life important ?,no,why is life important ?,why is life important ?,why is life important ?,life important ?
4,FYC0FTFB,How could I be helped to go through the depres...,Depression,How could I be helped to go through the depre...,yes,how could i be helped to go through the depre...,how could i be helped to go through the depre...,how could i be helped to go through the depres...,could helped go depression ?


In [24]:
train['Changed'] = np.where(train['text_stop']==train['text_spellchecker'], 'no', 'yes')
print("{} of {} ({:.4f}%) questions have been changed.".format(len(train[train['Changed']=='yes']), 
                                    len(train), 100*len(train[train['Changed']=='yes'])/len(train)))

test['Changed'] = np.where(test['text_stop']==test['text_spellchecker'], 'no', 'yes')
print("{} of {} ({:.4f}%) questions have been changed.".format(len(test[test['Changed']=='yes']), 
                                    len(test), 100*len(test[test['Changed']=='yes'])/len(test)))

599 of 616 (97.2403%) questions have been changed.
304 of 309 (98.3819%) questions have been changed.


In [43]:
# test['text_processed'].values

In [26]:
from collections import Counter
cnt = Counter()
for text in train["text_stop"].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(10)

[('?', 209),
 ('feel', 141),
 ('alcohol', 84),
 (',', 76),
 ('stop', 54),
 ('depression', 52),
 ('life', 50),
 ('like', 39),
 ('depressed', 27),
 ('avoid', 26)]

In [28]:
FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
print(f"FREQWORDS : {FREQWORDS}")
def remove_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

train["text_wo_stopfreq"] = train["text_stop"].apply(lambda text: remove_freqwords(text))
test["text_wo_stopfreq"] = test["text_stop"].apply(lambda text: remove_freqwords(text))

train.head()

FREQWORDS : {'feel', 'stop', 'depression', ',', 'life', '?', 'alcohol', 'avoid', 'like', 'depressed'}


,ID,text,label,text_processed,Changed,text_lower,text_contraction,text_spellchecker,text_stop,text_wo_stopfreq
0,SUAVK39Z,I feel that it was better I dieAm happy,Depression,I feel that it was better I die Am happy,yes,i feel that it was better i die am happy,i feel that it was better i die am happy,i feel that it was better i die am happy,feel better die happy,better die happy
1,9JDAGUV3,Why do I get hallucinations?,Drugs,Why do I get hallucinations ?,yes,why do i get hallucinations ?,why do i get hallucinations ?,why do i get hallucinations ?,get hallucinations ?,get hallucinations
2,419WR1LQ,I am stresseed due to lack of financial suppor...,Depression,I am stresseed due to lack of financial suppor...,yes,i am stresseed due to lack of financial suppor...,i am stresseed due to lack of financial suppor...,i am stressed due to lack of financial support...,stressed due lack financial support school,stressed due lack financial support school
3,6UY7DX6Q,Why is life important?,Suicide,Why is life important ?,yes,why is life important ?,why is life important ?,why is life important ?,life important ?,important
4,FYC0FTFB,How could I be helped to go through the depres...,Depression,How could I be helped to go through the depre...,yes,how could i be helped to go through the depre...,how could i be helped to go through the depre...,how could i be helped to go through the depres...,could helped go depression ?,could helped go


In [29]:
train['Changed'] = np.where(train['text_wo_stopfreq']==train['text_stop'], 'no', 'yes')
print("{} of {} ({:.4f}%) questions have been changed.".format(len(train[train['Changed']=='yes']), 
                                    len(train), 100*len(train[train['Changed']=='yes'])/len(train)))

test['Changed'] = np.where(test['text_wo_stopfreq']==test['text_stop'], 'no', 'yes')
print("{} of {} ({:.4f}%) questions have been changed.".format(len(test[test['Changed']=='yes']), 
                                    len(test), 100*len(test[test['Changed']=='yes'])/len(test)))

468 of 616 (75.9740%) questions have been changed.
233 of 309 (75.4045%) questions have been changed.


In [30]:
# Drop the two columns which are no more needed 
# df.drop(["text_wo_punct", "text_wo_stop"], axis=1, inplace=True)

n_rare_words = 10
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
print(f"RAREWORDS : {RAREWORDS}")

def remove_rarewords(text):
    """custom function to remove the rare words"""
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

train["text_wo_stopfreqrare"] = train["text_wo_stopfreq"].apply(lambda text: remove_rarewords(text))
test["text_wo_stopfreqrare"] = test["text_wo_stopfreq"].apply(lambda text: remove_rarewords(text))

train.head()

RAREWORDS : {'despising', 'become', 'first', 'later', 'energy', 'oneself', 'daily', 'remedy', 'ask', 'migraines'}


,ID,text,label,text_processed,Changed,text_lower,text_contraction,text_spellchecker,text_stop,text_wo_stopfreq,text_wo_stopfreqrare
0,SUAVK39Z,I feel that it was better I dieAm happy,Depression,I feel that it was better I die Am happy,yes,i feel that it was better i die am happy,i feel that it was better i die am happy,i feel that it was better i die am happy,feel better die happy,better die happy,better die happy
1,9JDAGUV3,Why do I get hallucinations?,Drugs,Why do I get hallucinations ?,yes,why do i get hallucinations ?,why do i get hallucinations ?,why do i get hallucinations ?,get hallucinations ?,get hallucinations,get hallucinations
2,419WR1LQ,I am stresseed due to lack of financial suppor...,Depression,I am stresseed due to lack of financial suppor...,no,i am stresseed due to lack of financial suppor...,i am stresseed due to lack of financial suppor...,i am stressed due to lack of financial support...,stressed due lack financial support school,stressed due lack financial support school,stressed due lack financial support school
3,6UY7DX6Q,Why is life important?,Suicide,Why is life important ?,yes,why is life important ?,why is life important ?,why is life important ?,life important ?,important,important
4,FYC0FTFB,How could I be helped to go through the depres...,Depression,How could I be helped to go through the depre...,yes,how could i be helped to go through the depre...,how could i be helped to go through the depre...,how could i be helped to go through the depres...,could helped go depression ?,could helped go,could helped go


In [48]:
# RAREWORDS

In [31]:
train['Changed'] = np.where(train['text_wo_stopfreq']==train['text_wo_stopfreqrare'], 'no', 'yes')
print("{} of {} ({:.4f}%) questions have been changed.".format(len(train[train['Changed']=='yes']), 
                                    len(train), 100*len(train[train['Changed']=='yes'])/len(train)))

test['Changed'] = np.where(test['text_wo_stopfreqrare']==test['text_wo_stopfreq'], 'no', 'yes')
print("{} of {} ({:.4f}%) questions have been changed.".format(len(test[test['Changed']=='yes']), 
                                    len(test), 100*len(test[test['Changed']=='yes'])/len(test)))

7 of 616 (1.1364%) questions have been changed.
3 of 309 (0.9709%) questions have been changed.


In [32]:
train.head()

,ID,text,label,text_processed,Changed,text_lower,text_contraction,text_spellchecker,text_stop,text_wo_stopfreq,text_wo_stopfreqrare
0,SUAVK39Z,I feel that it was better I dieAm happy,Depression,I feel that it was better I die Am happy,no,i feel that it was better i die am happy,i feel that it was better i die am happy,i feel that it was better i die am happy,feel better die happy,better die happy,better die happy
1,9JDAGUV3,Why do I get hallucinations?,Drugs,Why do I get hallucinations ?,no,why do i get hallucinations ?,why do i get hallucinations ?,why do i get hallucinations ?,get hallucinations ?,get hallucinations,get hallucinations
2,419WR1LQ,I am stresseed due to lack of financial suppor...,Depression,I am stresseed due to lack of financial suppor...,no,i am stresseed due to lack of financial suppor...,i am stresseed due to lack of financial suppor...,i am stressed due to lack of financial support...,stressed due lack financial support school,stressed due lack financial support school,stressed due lack financial support school
3,6UY7DX6Q,Why is life important?,Suicide,Why is life important ?,no,why is life important ?,why is life important ?,why is life important ?,life important ?,important,important
4,FYC0FTFB,How could I be helped to go through the depres...,Depression,How could I be helped to go through the depre...,no,how could i be helped to go through the depre...,how could i be helped to go through the depre...,how could i be helped to go through the depres...,could helped go depression ?,could helped go,could helped go


In [15]:
import ipdb

In [16]:
lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
#     ipdb.set_trace()
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

# train["text_lemmatized"] = train["text_spellchecker"].apply(lambda text: lemmatize_words(text))
# test["text_lemmatized"] = test["text_spellchecker"].apply(lambda text: lemmatize_words(text))

# train["text_lemmatized"] = train["text_wo_stopfreqrare"].apply(lambda text: lemmatize_words(text))
# test["text_lemmatized"] = test["text_wo_stopfreqrare"].apply(lambda text: lemmatize_words(text))

# train.head()

In [18]:
lemmatizer.lemmatize("talking")

'talking'

In [22]:
# nltk.stem??
# ("talking")

In [17]:
lemmatize_words("talking")

> <ipython-input-16-c4500ae1ca42>(6)lemmatize_words()
      5     ipdb.set_trace()
----> 6     return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])
      7 



ipdb>  pos_tagged_text


[('talking', 'VBG')]


ipdb>  q


BdbQuit: 

In [34]:
train['Changed'] = np.where(train['text_wo_stopfreqrare']==train['text_lemmatized'], 'no', 'yes')
print("{} of {} ({:.4f}%) questions have been changed.".format(len(train[train['Changed']=='yes']), 
                                    len(train), 100*len(train[train['Changed']=='yes'])/len(train)))

test['Changed'] = np.where(test['text_wo_stopfreqrare']==test['text_lemmatized'], 'no', 'yes')
print("{} of {} ({:.4f}%) questions have been changed.".format(len(test[test['Changed']=='yes']), 
                                    len(test), 100*len(test[test['Changed']=='yes'])/len(test)))

318 of 616 (51.6234%) questions have been changed.
175 of 309 (56.6343%) questions have been changed.


In [35]:
# train['Changed'] = np.where(train['text_spellchecker']==train['text_lemmatized'], 'no', 'yes')
# print("{} of {} ({:.4f}%) questions have been changed.".format(len(train[train['Changed']=='yes']), 
#                                     len(train), 100*len(train[train['Changed']=='yes'])/len(train)))

# test['Changed'] = np.where(test['text_spellchecker']==test['text_lemmatized'], 'no', 'yes')
# print("{} of {} ({:.4f}%) questions have been changed.".format(len(test[test['Changed']=='yes']), 
#                                     len(test), 100*len(test[test['Changed']=='yes'])/len(test)))

# Save processed data

In [36]:
train.head()

,ID,text,label,text_processed,Changed,text_lower,text_contraction,text_spellchecker,text_stop,text_wo_stopfreq,text_wo_stopfreqrare,text_lemmatized
0,SUAVK39Z,I feel that it was better I dieAm happy,Depression,I feel that it was better I die Am happy,yes,i feel that it was better i die am happy,i feel that it was better i die am happy,i feel that it was better i die am happy,feel better die happy,better die happy,better die happy,well die happy
1,9JDAGUV3,Why do I get hallucinations?,Drugs,Why do I get hallucinations ?,yes,why do i get hallucinations ?,why do i get hallucinations ?,why do i get hallucinations ?,get hallucinations ?,get hallucinations,get hallucinations,get hallucination
2,419WR1LQ,I am stresseed due to lack of financial suppor...,Depression,I am stresseed due to lack of financial suppor...,yes,i am stresseed due to lack of financial suppor...,i am stresseed due to lack of financial suppor...,i am stressed due to lack of financial support...,stressed due lack financial support school,stressed due lack financial support school,stressed due lack financial support school,stress due lack financial support school
3,6UY7DX6Q,Why is life important?,Suicide,Why is life important ?,no,why is life important ?,why is life important ?,why is life important ?,life important ?,important,important,important
4,FYC0FTFB,How could I be helped to go through the depres...,Depression,How could I be helped to go through the depre...,yes,how could i be helped to go through the depre...,how could i be helped to go through the depre...,how could i be helped to go through the depres...,could helped go depression ?,could helped go,could helped go,could help go


In [40]:
train['text'] = train['text_spellchecker']
test['text'] = test['text_spellchecker']

proccessed_train = train[['ID', 'text', 'label']]
proccessed_test = test[['ID', 'text']]

proccessed_train.head()

,ID,text,label
0,SUAVK39Z,i feel that it was better i die am happy,Depression
1,9JDAGUV3,why do i get hallucinations ?,Drugs
2,419WR1LQ,i am stressed due to lack of financial support...,Depression
3,6UY7DX6Q,why is life important ?,Suicide
4,FYC0FTFB,how could i be helped to go through the depres...,Depression


In [41]:
proccessed_train.to_csv('./data/Train_process.csv', index=False)
proccessed_test.to_csv('./data/Test_process.csv', index=False)

In [42]:
proccessed_test.tail()

,ID,text
304,Z9A6ACLK,yes
305,ZDUOIGKN,my girlfriend dumped me
306,ZHQ60CCH,how can i go back to being my old self ?
307,ZVIJMA4O,is it true hang is medicinal ?
308,ZYIFAY98,how can i overcome the problem ?
